# _1_. 동의어에 대해 Embedding Vector 수치 확인

> 유의어, 동의어 구별이 안 된다. 당연히 똑같다.

* 동의어 있는 문장 확인
    - vocab_dict 생성
    - decode 이용해서 동의어 있나 확인
    - 동의어 있는 문장 인덱스 저장

* Functional API로 임베딩 레이어 구성

* Embedding latent feature 추출하여 임베딩된 수치 확인 

> 강사님 조언
- `predict` 해서 확인하면 힘들다. 그리고 애초에 똑같은 게 맞다.
- `weight` 확인해 보자.

In [8]:
# 모듈 불러오기

import random

from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Bidirectional, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers

In [2]:
# IMDB 데이터셋 문장 decode
def decode_sent(sentences):
    '''
    - 0: padding, 1: start, 2: OOV
    - 실제 word index에서 3을 빼고, 없으면 '*'로 채운다.
    '''
    temp_sent = []
    for sent in sentences:
        temp = imdb_vocab_dict.get(sent-3, '*')
        temp_sent.append(temp)
    
    comb_sent = " ".join(temp_sent)

    return comb_sent

In [3]:
# 아무 단어나 문단에서 찾아오는 함수
def check_syn(sentences, dictionary, threshold, k=10):

    syn_idx = random.randint(3, max_features) # 숫자: 찾을 동의어 인덱스.
    synonym = dictionary[syn_idx]
    print(f"찾을 단어 인덱스: {syn_idx}, 찾을 단어: {synonym}") # 확인용

    common_sentences = []

    cnt = 0
    for i in range(len(sentences)):
        if cnt == k:
            break
        sent = sentences[i]
        decoded_sent = decode_sent(sent)        
        if synonym in decoded_sent:
            print(f"{i}번째 문장:\n    {decoded_sent}") # 확인용
            cnt += 1
            common_sentences.append(i)

    return common_sentences

In [4]:
# 문장 길이 체크
def check_len(m, sentences):
    cnt = 0
    for sent in sentences:
        if len(sent) <= m:
            cnt += 1
    
    return f'전체 문장 중 길이가 {m} 이하인 샘플의 비율: {(cnt/len(sentences))*100}'

In [5]:
# 데이터 로드
max_features = int(input('최대 단어 빈도 설정: '))
(X_train_raw, y_train), (X_test_raw, y_test) = imdb.load_data(num_words=max_features)

# 원본 데이터 보존 및 데이터 준비
X_train = X_train_raw.copy()
X_test = X_test_raw.copy()
print(f"훈련 데이터: {X_train.shape}, 훈련 라벨: {y_train.shape}")
print(f"테스트 데이터: {y_train.shape}, 테스트 라벨: {y_test.shape}")
print()

# 어휘 사전
vocabulary = imdb.get_word_index()
imdb_vocab_dict = dict((v, k) for k, v in vocabulary.items())

# 문장 패딩 길이 설정
for length in range(100, 1000, 50):
    print(check_len(length, X_train))
print()
max_length = int(input('문장 패딩 길이 설정: '))

# 패딩
X_train = pad_sequences(X_train, maxlen=max_length)
X_test = pad_sequences(X_test, maxlen=max_length)

# 모델 파라미터 설정
BATCH = int(input('배치 사이즈 설정: '))
EMBED_DIM = int(input('임베딩 차원 설정: '))
n_hidden = int(input('은닉 노드 수 설정: '))
EPOCHS = int(input('학습 횟수 설정: '))

# 모델 레이어 설정 및 구성
X_input = Input(batch_shape=(None, max_length)) # 시퀀스 길이만큼 들어간다
X_embed = Embedding(input_dim=max_features, output_dim=EMBED_DIM, input_length=max_length)(X_input)
X_lstm = Bidirectional(LSTM(n_hidden))(X_embed) # many to one이므로 return sequences 없음
y_output = Dense(1, activation='sigmoid')(X_lstm)

# 모델 구성
model = Model(X_input, y_output)
embed_model = Model(X_input, X_embed) ### 이

# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.002))
print("========== 모델 전체 구조 확인 ==========")
print(model.summary())
print()

# 임베딩 모델 확인
print("========== 임베딩 모델 구조 확인 ==========")
print(embed_model.summary())
print()

# 모델 훈련
es = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
hist = model.fit(X_train, y_train,
                 batch_size=BATCH,
                 epochs=EPOCHS,
                 validation_data=(X_test, y_test),
                 callbacks=[es])

# 임베딩 latent feature 확인
embedded_sentences = embed_model.predict(X_train)
print(f"임베딩된 문장: {embedded_sentences.shape}")
print()

# 동의어 찾기
synonyms = check_syn(X_train, imdb_vocab_dict, max_features)
print()

# 동음이의어 있는 문장 확인: '*'이 많아서 문장으로만 보면 다 이렇게 보인다.
for idx in synonyms:
    print(decode_sent(X_train_raw[idx]))
    print(embedded_sentences[idx, :, :])
    print()

최대 단어 빈도 설정: 6000
17465344/17464789 [==============================] - 0s 0us/step
훈련 데이터: (25000,), 훈련 라벨: (25000,)
테스트 데이터: (25000,), 테스트 라벨: (25000,)

1646592/1641221 [==============================] - 0s 0us/step
전체 문장 중 길이가 100 이하인 샘플의 비율: 11.288
전체 문장 중 길이가 150 이하인 샘플의 비율: 37.732
전체 문장 중 길이가 200 이하인 샘플의 비율: 57.292
전체 문장 중 길이가 250 이하인 샘플의 비율: 68.688
전체 문장 중 길이가 300 이하인 샘플의 비율: 76.36800000000001
전체 문장 중 길이가 350 이하인 샘플의 비율: 81.93599999999999
전체 문장 중 길이가 400 이하인 샘플의 비율: 86.064
전체 문장 중 길이가 450 이하인 샘플의 비율: 89.184
전체 문장 중 길이가 500 이하인 샘플의 비율: 91.56800000000001
전체 문장 중 길이가 550 이하인 샘플의 비율: 93.308
전체 문장 중 길이가 600 이하인 샘플의 비율: 94.812
전체 문장 중 길이가 650 이하인 샘플의 비율: 95.92399999999999
전체 문장 중 길이가 700 이하인 샘플의 비율: 96.72800000000001
전체 문장 중 길이가 750 이하인 샘플의 비율: 97.432
전체 문장 중 길이가 800 이하인 샘플의 비율: 98.012
전체 문장 중 길이가 850 이하인 샘플의 비율: 98.5
전체 문장 중 길이가 900 이하인 샘플의 비율: 98.832
전체 문장 중 길이가 950 이하인 샘플의 비율: 99.136

문장 패딩 길이 설정: 700
배치 사이즈 설정: 100
임베딩 차원 설정: 64
은닉 노드 수 설정: 128
학습 횟수 설정: 1000
========== 모델 전체 구조 확인

In [20]:
# 가중치 행렬 확인
W_embed = model.layers[1].get_weights()[0]
print(W_embed.shape)
print(W_embed)

(6000, 64)
[[-0.18629795 -0.33082274 -0.04060129 ...  0.19987082  0.0543128
   0.12141238]
 [-0.04181389 -0.45165324 -0.17377338 ...  0.37707064  0.08209425
  -0.04206326]
 [-0.0329046  -0.1082665  -0.05962329 ...  0.09577476 -0.00636604
   0.01586349]
 ...
 [ 0.02628802  0.04183222  0.06038851 ... -0.07766243 -0.07020175
  -0.01413197]
 [ 0.13479842  0.14979705 -0.12941438 ...  0.07325806  0.19518562
  -0.2134134 ]
 [-0.06347922  0.00468148  0.03113244 ...  0.07939062  0.00450522
   0.05160246]]


# 문제 추가: 미완

- `father`, `mother` 유사도 측정.
- 가중치 행렬 확인.


In [24]:
list(imdb_vocab_dict.values()).index('father')

6040

In [25]:
list(imdb_vocab_dict.values()).index('mother')

74809